# Whisper for OpenVINO GenAI 單獨執行含 Gradio 介面範例(deepseek-r1-run.ipynb)
by Jack OmniXRI, 2025/03/24

原始範例: https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/whisper-asr-genai/whisper-asr-genai.ipynb

原始範例全部執行(Restart Kernel and Run All Cells）後，預設會取得 whisper-tiny 模型， 經轉換後會得到  whisper-tiny-quantized 量化後較小的模型。如果辨識率不佳想要使用大一點的模型，可採用手動單步執行方式完成。

本範例為簡化版，執行前需先完整執行過原始範例，取得模型並轉換好 OpenVINO 所需 IR 格式。

## 1. 設定工作參數

In [1]:
# 指定執行裝置 "CPU", "GPU", "NPU", "AUTO"
device = "GPU"

# 指定模型名稱  "whisper-large-v3-turbo", "whisper-large-v3", "whisper-large-v2", "whisper-large",
#              "whisper-medium", "whisper-small", "whisper-base", "whisper-tiny",
# 若要指定量化後模型 model_id 則在原名稱後方再加上 -quantized
model_id = "whisper-small"

In [2]:
# 指定 OpenVINO IR 格式模型(*.xml)、權重(*.bin) 路徑，假設對應模型已下載並轉檔完成

from pathlib import Path

model_path = Path(f"{model_id}")

## 2. 使用 OpenVINO GenAI 實例化 LLM 流水線

In [3]:
import openvino_genai as ov_genai
import sys
import time

print(f"Loading model from {model_path}\n")

t0 = time.perf_counter() # 取得開始時間

ov_pipe = ov_genai.WhisperPipeline(str(model_path), device)

t1 = time.perf_counter() # 取得結束時間
print(f"Model loading time: {(t1-t0):.4f} seconds\n") # 列出模型載入時間

Loading model from whisper-small



C:\openvino_env_py310\lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


Model loading time: 2.0871 seconds



Let's run the `transcribe` task. We just call `generate` for that and put array as input.

## 3. 啟動聊天機器人人機介面

Gradio 人機介面設定可參考 gradio_helper_whisper_run.py

執行後可直接在欄位上操作，亦可開啟網址 http://127.0.0.1:7860 (http://localhost:7860) 使用瀏覽器頁面進行操作。

操作時可點選下方範例聲音聲案或自行選擇聲音檔案載入，或點擊麥克風圖案，再點擊紅色圓點錄音符號進行錄音，接著按下「Transcribe」即可將聲音轉成文字，同時顯示轉換耗時及文字內容，目前輸人的語音可支援中英混合。若想將中文即時翻成英文，則按下「Translate」就會把結果顯示在「Result」區域。

In [ ]:
from gradio_helper_whisper_run import make_demo, GradioPipeline

pipe = ov_pipe

gr_pipeline = GradioPipeline(pipe, model_id, quantized=False)

demo = make_demo(gr_pipeline)

try:
    demo.launch(debug=True)
except Exception:
    demo.launch(share=True, debug=True)
# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
